In [1]:
import pandas as pd
import numpy as np
from R125_bmimPF6_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = 0.3
    init_kappa_1_2A = 0.05
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimPF6'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimPF6','bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'bmimPF6'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data['x_bmimPF6']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimPF6', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'bmimPF6']"]

#     pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

#     obj_value, parameters,a= pest.theta_est(calc_cov=True)

In [3]:
data = pd.read_csv('R125IL.csv')
print(data)

    temperature  pressure  x_R125  x_bmimPF6
0         283.1      9900   0.013      0.987
1         283.2     99600   0.034      0.966
2         283.3    199600   0.075      0.925
3         283.2    299600   0.126      0.874
4         283.3    399600   0.185      0.815
5         283.1    499700   0.255      0.745
6         283.2    599700   0.342      0.658
7         283.1    699600   0.462      0.538
8         298.2     99600   0.023      0.977
9         298.2    199800   0.050      0.950
10        298.1    299500   0.080      0.920
11        298.2    399800   0.111      0.889
12        298.2    499500   0.143      0.857
13        298.2    599500   0.180      0.820
14        298.2    799600   0.261      0.739
15        298.1    999800   0.363      0.637
16        323.1     99600   0.013      0.987
17        323.1    199800   0.028      0.972
18        323.2    300000   0.042      0.958
19        323.1    399700   0.057      0.943
20        323.2    499900   0.072      0.928
21        

In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.09 #mole fraction

for i in range(N):
    print(i)

    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()

    data_["x_bmimPF6"] += data_["x_bmimPF6"] * noise_x
    data_.loc[data_['x_bmimPF6'] <= 0.0001, 'x_bmimPF6'] = 0.0001
    data_.loc[data_['x_bmimPF6'] >= 0.9999 , 'x_bmimPF6'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.09619061749224725
2022-03-04 20:27:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04

2022-03-04 20:29:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:57 [INFO] idaes

2022-03-04 20:32:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:32:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:32:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:32:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:32:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:32:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:32:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:32:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:32:17 [INFO] idaes

2022-03-04 20:33:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:49 [INFO] idaes

2022-03-04 20:35:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:35:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:35:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:35:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:35:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:35:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:35:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:35:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:35:27 [INFO] idaes

...

    model.name="_Q_opt";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
        Exceeded.
fs.properties.PR_kappa_A[bmimPF6,R125] = 0.31261392106430796
fs.properties.PR_kappa_A[R125,bmimPF6] = 0.08208545264995873
92
0.02849458731621282
2022-03-04 21:40:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:40:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:40:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:40:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:40:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:40:21 [INFO] idaes.init.fs.state_block: Property package initialization: 

2022-03-04 21:41:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:41:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimPF6,R125] = 0.021626595633729595
fs.properties.PR_kappa_A[R125,bmimPF6] = 0.03479752682491512
94
0.024476170925140525
2022-03-04 21:41:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:41:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:41:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:41:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:41:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-0

2022-03-04 21:42:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:42:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimPF6,R125] = 0.1796833887082618
fs.properties.PR_kappa_A[R125,bmimPF6] = 0.04735416339731325
96
0.035056480399950345
2022-03-04 21:42:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:42:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:42:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:42:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:42:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-

2022-03-04 21:43:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:43:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimPF6,R125] = 0.37143527906425333
fs.properties.PR_kappa_A[R125,bmimPF6] = 0.057734520769014225
98
0.12240119938762924
2022-03-04 21:43:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:43:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:43:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:43:54 [INFO] idaes.init.fs.state_block: Proper

2022-03-04 21:44:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:44:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:44:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:44:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimPF6,R125] = 0.11849318312759013
fs.properties.PR_kappa_A[R125,bmimPF6] = 0.04273373418867198


In [5]:
print(results_params)

[[ 0.37806669  0.05711474]
 [ 0.20344804  0.04907598]
 [ 0.37729919  0.05717779]
 [ 1.67791541 -0.2358135 ]
 [ 0.28553383  0.05474168]
 [ 0.19921226  0.04877191]
 [ 0.37208469  0.05762404]
 [ 0.41196464  0.05494617]
 [ 0.35139542  0.05963809]
 [ 0.34918286  0.05886304]
 [ 0.33055837  0.05767987]
 [ 0.21986383  0.05024311]
 [ 0.37491659  0.05737887]
 [ 0.23497814  0.05130218]
 [ 0.37158543  0.05770974]
 [ 0.43928448  0.02429448]
 [ 0.37897217  0.05704434]
 [ 0.35377392  0.05971781]
 [ 0.3650355   0.0582926 ]
 [ 0.32994841  0.05764081]
 [ 0.16597465  0.04634301]
 [ 0.24522933  0.05201222]
 [ 0.3591524   0.05931426]
 [ 0.38481511  0.05660204]
 [ 0.38236616  0.0567807 ]
 [ 0.35993024  0.05891256]
 [ 0.05695628  0.03778791]
 [ 0.16132521  0.045997  ]
 [ 0.21094173  0.0136309 ]
 [ 0.36242888  0.05863464]
 [ 0.28959394  0.05501131]
 [ 0.35606072  0.0594936 ]
 [        nan         nan]
 [ 0.37227457  0.05759486]
 [ 0.37782102  0.0571347 ]
 [ 0.32732723  0.05747273]
 [ 0.38739498  0.0564107 ]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_bmimPF6_09_2.csv')